In [5]:
import urllib
import requests
from pandas.io.json import json_normalize
import json
import pandas as pd
import datetime
import time
import math
import numpy as np
from node import Node
from edge import Edge
from neural_network import NeuralNetwork

In [ ]:
url = 'https://rsbuddy.com/exchange/summary.json'
urljson = urllib.request.urlopen(url)
jsondata = json.loads(urljson.read().decode())

In [ ]:
#Get a list of all items
df = pd.read_json(path_or_buf='https://rsbuddy.com/exchange/summary.json',orient='index', convert_axes=True)
df = df[['id','name','buy_average','buy_quantity','sell_average','sell_quantity','overall_average','overall_quantity']]
data = df.sort_values(by=['id']).reset_index()
data = data.drop(labels='index',axis=1)

In [ ]:
#Run this to populate a csv: itemKey
itemKey = data[['id', 'name']]
itemKey.to_csv(path_or_buf='./itemKey.csv', columns=('id','name'), index=False)

In [ ]:
#Create test set of specified range number of items
data = pd.read_csv('./itemKey.csv', skiprows=[])
startNum = 0
numItems = 3
inputdata = data[startNum:startNum+numItems]
inputdata

In [ ]:
#Use this to pull a specific items by name
names = ['Armadyl godsword','Fire rune', 'Soft clay']

#read the itemKey csv and parse the key for associated item ids
data = pd.read_csv('./itemKey.csv', skiprows=[])
inputdata = pd.DataFrame()
for name in names:
    inputdata = inputdata.append(data.loc[data['name'] == name])
inputdata = inputdata.reset_index().drop(labels='index',axis=1)

print("Query List:")
print(inputdata)

In [ ]:
i = 0
firtIteration = True
urlquery = pd.DataFrame(columns=['id', 'name', 'data'])
#Iterate through all items, grab data from api and append to urlquery dataframe
while(i<inputdata['id'].count()):
    key = inputdata.iloc[i]['id']
    name = inputdata.iloc[i]['name']
    if (firtIteration == True):
        print("Item Id - " + str(key) + ": " + name)
    #Attempt to get data from API, retry if HTTP error
    try:
        url = f'https://api.rsbuddy.com/grandExchange?a=graph&g=240&start=1474615279000&i={key}'
        tempDataFrame = pd.DataFrame()
        tempDataFrame = pd.read_json(path_or_buf=url,orient='records', convert_axes=False)
        urlquery = urlquery.append({'id':key, 'name': name, 'data':tempDataFrame}, ignore_index=True)
        i+=1
        #check if there are more items to run. If no more items, return message indicating the process is done
        if (i == inputdata['id'].count()):
            print("It worked! That was the last item!")
        else:
            print("It worked! On to the next one.")
        firtIteration = True
    except:
        print("Got fucked. Trying again. :)")
        time.sleep(1)
        firtIteration = False
#Reformat urlquery columns, convert timestamp from miliseconds to seconds, convert timestamp to date, and sort by date
for index, row in urlquery.iterrows():
    row['data'] = row['data'][['ts','buyingPrice','buyingCompleted','sellingPrice','sellingCompleted','overallPrice','overallCompleted']]
    row['data'] = row['data'].sort_values(by=['ts'],ascending=1).reset_index()
    count = 0 
    for ind, r in row['data'].iterrows():
        r['ts'] = r['ts']/1000
        row['data'].loc[ind, 'ts'] = int(r['ts'])
        row['data'].loc[ind, 'date'] = datetime.datetime.fromtimestamp(r['ts']).isoformat()
    row['data'] = row['data'].drop(labels='index',axis=1)

In [ ]:
test_set_size = 360 #In Days - define the number of epochs to include in the test data

#create test_item object to log training and test sets
test_item = pd.DataFrame(columns=['id', 'name', 'train_x', 'train_y', 'test_x', 'test_y', 'pred_y'])
h = 0
#iterate through the urlquery results to generate training sets
while(h<urlquery['id'].count()):  
    item_id = urlquery.iloc[h]['id'] #Define the item to be queried for
    print("Item ID: " + str(item_id))
    #takes the item_id and generates the test data for the specified parameters as an array
    test_set = urlquery.loc[urlquery['id'] == item_id]
    train_x_headers = list(urlquery.iloc[0]['data'].columns.values[0:5])
    
    #for column key, uncomment this line below:
    #print(train_x_headers)

    #create training datasets
    train_x = test_set.iloc[0]['data'].iloc[0:math.ceil(test_set_size*6)].values[:,0:5]
    train_y = test_set.iloc[0]['data'].iloc[0:math.ceil(test_set_size*6)].values[:,5]
    print ("The shape of the " + test_set.iloc[0]['name'] +  " input array is: "+ str(train_x.shape))

    #create test set to estimate next 6 prices (next day of prices)
    test_x = test_set.iloc[0]['data'].iloc[math.ceil(test_set_size*6):math.ceil(test_set_size*6)+6].values[:,0:5]

    #test key to compare to Y-hat
    test_y = test_set.iloc[0]['data'].iloc[math.ceil(test_set_size*6):math.ceil(test_set_size*6)+6].values[:,5]
    test_item = test_item.append({'id':item_id, 'name': test_set.iloc[0]['name'], 'train_x':train_x, 'train_y':train_y, 'test_x':test_x, 'test_y': test_y}, ignore_index=True)
    print("Record completed at test_item position [" + str(h) + "]")
    h += 1
    #TODO: Get this all in a for loop to iterate over multiple entries

In [ ]:
#Test the creation of a neural network
nn = NeuralNetwork()
nn.generateInitialNetwork([1,1,1])
nn.calculateNodeActivations()
print(nn.outputNode.getActivation())



<font size="+3"><b>Testing of network size:</b></font>

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#import keras.backend as K
train_x = test_item.iloc[2]['train_x'] # test data. not real
 
def create_model(layesr,activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=train_x.shape[1]))
            model.add(Activation(activation))
        else: 
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(1)) #Note: no activations present beyond this point

    model.compile(optimizer='adadelta', loss='mse')
    return model

model = KerasRegressor(build_fn=create_model, verbose = 0)

In [ ]:
layers = [[16], [4,2], [4], [16,4]]
activations = [tanh, relu]
param_grid = dict(layers=layers, activation=activations, batch_size = [42, 180], epochs[6])
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error')

In [25]:
import urllib
import requests
from pandas.io.json import json_normalize
import json
import pandas as pd
import datetime
import time
import math
import numpy as np

from neural_network import NeuralNetwork

"""
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Activation, Dense
from keras.models import Sequential
"""


def generate_item_records_from_summary():
    """ Generate item records using the summary.json link from the OSBuddy API """

    df = pd.read_json(path_or_buf='https://rsbuddy.com/exchange/summary.json',orient='index', convert_axes=True)
    df = df[['id','name','buy_average','buy_quantity','sell_average','sell_quantity','overall_average','overall_quantity']]
    data = df.sort_values(by=['id']).reset_index()
    data = data.drop(labels='index',axis=1)

    #Output item id/name pairs to a csv file
    item_key = data[['id', 'name']]
    file_name = './item_key.csv'
    item_key.to_csv(path_or_buf=file_name, columns=('id','name'), index=False)


def generate_input_data_by_item_number(start_num, num_items): 
    """Pull items by range of Index values"""
    file_name = './item_key.csv'
    all_items = pd.read_csv(file_name, skiprows=[])
    items = all_items[start_num: start_num + num_items]
    print("Item List generated by item number:")
    print(items)
    return items

def generate_input_data_by_item_name(names):  #Pull a specific items by name
    """Pull specific items by name"""
    file_name = './item_key.csv'
    data = pd.read_csv(file_name, skiprows=[])
    items = pd.DataFrame()
    for name in names:
        items = items.append(data.loc[data['name'] == name])
    items = items.reset_index().drop(labels='index',axis=1)
    print("Item List generated by name:")
    print(items)
    return items

def get_item_records_from_url(input_data, test_set_size):
    """ Iterate through all items, grab data from api and append to dataframe """
    i = 0
    item_lookup_failed = False
    item_records = pd.DataFrame(columns=['id', 'name', 'data'])
    while(i<input_data['id'].count()):
        key = input_data.iloc[i]['id']
        name = input_data.iloc[i]['name']
        #Print item information only on first data retrieval attempt 
        if not item_lookup_failed:
            print('Querying API for ' + name + ' data. ' + 'Item Id: ' + str(key))
        try:
            #Attempt to get data from API, retry if HTTP error
            url = f'https://api.rsbuddy.com/grandExchange?a=graph&g=240&start=1474615279000&i={key}'
            temp_df = pd.DataFrame()
            temp_df = pd.read_json(path_or_buf=url,orient='records', convert_axes=False)
            item_records = item_records.append({'id':key, 'name': name, 'data':temp_df}, ignore_index=True)
            i+=1
            item_lookup_failed = False
        except:
            print("Retrying...")
            time.sleep(1) #Avoid getting blacklisted by API
            item_lookup_failed = True
    print('Item retrieval complete!')

    #Add correct formatting to item record dates/times
    item_records = format_item_record_dates(item_records)

    return item_records

def format_item_record_dates(item_records):
    """ Converts timestamp to Unix seconds, and tacks on formatted date field """

    print("Formatting item record dates...")
    for _, row in item_records.iterrows():
        row['data'] = row['data'][['ts','buyingPrice','buyingCompleted','sellingPrice','sellingCompleted', 'overallPrice','overallCompleted']]
        row['data'] = row['data'].sort_values(by=['ts'],ascending=1).reset_index()
        for ind, r in row['data'].iterrows():
            r['ts'] = r['ts']/1000
            row['data'].loc[ind, 'ts'] = int(r['ts'])
            row['data'].loc[ind, 'date'] = datetime.datetime.fromtimestamp(r['ts']).isoformat()
        row['data'] = row['data'].drop(labels='index',axis=1)
    return item_records

def generate_train_and_test_deltas(item_records):
    """ 
        Iterate through each item record and generate percentage deltas for buy price, 
        buy completed, sell price, sell completed, overall price, and overall completed
        compared to last sample period. 
    """
    for _, row in item_records.iterrows():
        for col_name in ['buyingPricePer', 'buyingCompletedPer', 'sellingPricePer', 
                        'sellingCompletedPer', 'overallPricePer', 'overallCompletedPer']:
            series_length = len(row['data']['ts'])
            row['data'][col_name] = pd.Series(np.random.randn(series_length), index=row['data'].index)
        #Each row represents and item, which has an associated dataframe 
        for index, _ in row['data'].iterrows():
            #Use data from last row to calc percentage change
            for col_name in ['buyingPrice', 'buyingCompleted', 'sellingPrice', 'sellingCompleted', 
                            'overallPrice', 'overallCompleted']:
                if(index == 0):
                    row['data'].loc[index, col_name + 'Per'] = 0
                else:
                    row['data'].loc[index, col_name + 'Per'] = calculate_percentage_change(row['data'], col_name, index)
                
                
            
def calculate_percentage_change(dataframe, columnName, index):
    """Calculate delta from last time stamp"""
    return (dataframe.loc[index, columnName] - dataframe.loc[index-1, columnName])/dataframe.loc[index-1, columnName]


def createTrainAndTestSet(item_records):
    print("Creating Test and Training Datasets...")
    #create test_item object to log training and test sets
    test_item = pd.DataFrame(columns=['id', 'name', 'train_val', 'test_val', 'train_per', 'test_per', 'pred_per'])
    record_index = 0
    #iterate through the urlquery results to generate training sets
    while(record_index < item_records['id'].count()):  
        item_id = item_records.iloc[record_index]['id'] 
        print("Creating Datasets for item ID: " + str(item_id))
        
        #takes the item_id and generates the test data for the specified parameters as an array
        test_set = item_records.loc[item_records['id'] == item_id]
        
        #for column key, uncomment this line below:
        #train_x_headers = list(urlquery.iloc[0]['data'].columns.values[0:6])
        #print(train_x_headers)

        #create training datasets
        train_val = test_set.iloc[0]['data'].iloc[0:math.ceil(test_set_size*6)].values[:,0:6]
        print ("The shape of the " + test_set.iloc[0]['name'] +  " input array is: "+ str(train_val.shape))

        #create test set to estimate next 6 prices (next day of prices)
        test_val = test_set.iloc[0]['data'].iloc[math.ceil(test_set_size*6):math.ceil(test_set_size*6)+6].values[:,0:6]

        #Generate deltas
        generate_train_and_test_deltas(item_records)

        #test key to compare to Y-hat
        test_item = 0
        #test_item = test_item.append({'id':item_id, 'name': test_set.iloc[0]['name'], 'train_val': train_val, 'test_val': test_val, 'train_per': train_per, 'test_per': test_per}, ignore_index=True)
        print("Record completed at test_item position [" + str(record_index) + "]")
        record_index += 1

    return test_item

if __name__ == '__main__':
    #Get a list of all items
    generate_item_records_from_summary()

    #Pull items by range of Index values
    input_data = generate_input_data_by_item_number(0,3)
    
    #Pull specific items by name
    #input_data = generate_input_data_by_item_name(['Armadyl godsword','Fire rune', 'Soft clay'])

    #In Days - define the number of epochs to include in the test data
    test_set_size = 30
    item_records = get_item_records_from_url(input_data, test_set_size)

    #Create test_item object to log training and test sets
    test_item = createTrainAndTestSet(item_records)

    #Test creation of Neural Net
    nn = NeuralNetwork()
    nn.generateInitialNetwork([1,1])
    nn.calculateNodeActivations()
    print(nn.outputNode.getActivation())

    #TODO: Create Year time stamp for seasonality
    #TODO: Create time from today time stamp
    #TODO: Create training and test sets
    #TODO: Test model size


    #random shit
    #import keras.backend as K
    """
    train_val = test_item.iloc[2]['train_val'] # test data. not real
    train_y = test_item.iloc[2]['train_y'] # test data. not real

    
    def create_model(layers,activation):
        model = Sequential()
        for i, nodes in enumerate(layers):
            if i==0:
                model.add(Dense(nodes,input_dim=train_val.shape[1]))
                model.add(Activation(activation))
            else: 
                model.add(Dense(nodes))
                model.add(Activation(activation))
        model.add(Dense(1)) #Note: no activations present beyond this point

        model.compile(optimizer='adadelta', loss='mse')
        return model

    print('creating model')
    model = KerasRegressor(build_fn=create_model, verbose = 0)
    layers = [[16], [4,2], [4], [16,4]]
    activations = ['tanh', 'relu']
    param_grid = dict(layers=layers, activation=activations, batch_size = [42, 180], epochs=[6])
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error')

    # grid_result = grid.fit(train_x, train_y) testing for network size """

Item List generated by item number:
   id          name
0   2    Cannonball
1   6   Cannon base
2   8  Cannon stand
Querying API for Cannonball data. Item Id: 2
Retrying...
Querying API for Cannon base data. Item Id: 6
Retrying...
Querying API for Cannon stand data. Item Id: 8
Item retrieval complete!
Formatting item record dates...
Creating Test and Training Datasets...
Creating Datasets for item ID: 2
The shape of the Cannonball input array is: (180, 6)
Record completed at test_item position [0]
Creating Datasets for item ID: 6
The shape of the Cannon base input array is: (180, 6)
Record completed at test_item position [1]
Creating Datasets for item ID: 8
The shape of the Cannon stand input array is: (180, 6)
Record completed at test_item position [2]
32


In [ ]:
item_records.iloc[0]['data']

In [ ]:
timestamps = pd.DataFrame(columns=['ts','tsFromCurrent', 'tsYtd'],data=item_records.iloc[0]['data']['ts'])
timestamps

In [ ]:
item_records.iloc[0]['data']['date']

In [ ]:
timestamps = pd.DataFrame(columns=['ts','tsFromCurrent', 'tsYtd'],data=item_records.iloc[0]['data']['ts'])
lastTs = timestamps['ts'][len(timestamps)-1]
i=0
while(i<len(timestamps)):
    timestamps['tsFromCurrent'][i] = timestamps['ts'][i] - lastTs
    i += 1
    



In [35]:
timestamps = pd.DataFrame(columns=['ts','tsFromCurrent', 'tsYtd'],data=item_records.iloc[0]['data']['ts'])
years = pd.DataFrame(columns=['year', 'unix'])

i = 0
for year in range(1970,2050):
    unix = (year - 1970) * 31557600
    years.loc[i] = [year,unix]
    i+=1

i = 0
for unixYear in years['unix']:
    if timestamps['ts'][i]<unixYear:
        while(timestamps['ts'][i]<unixYear):
            timestamps['tsYtd'][i] = timestamps['ts'][i] - unixYear
            i+=1
    else:
        i+=1


C:\Users\Jonathan Stav\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


KeyError: 3973

    year        unix
0   1970           0
1   1971    31557600
2   1972    63115200
3   1973    94672800
4   1974   126230400
5   1975   157788000
6   1976   189345600
7   1977   220903200
8   1978   252460800
9   1979   284018400
10  1980   315576000
11  1981   347133600
12  1982   378691200
13  1983   410248800
14  1984   441806400
15  1985   473364000
16  1986   504921600
17  1987   536479200
18  1988   568036800
19  1989   599594400
20  1990   631152000
21  1991   662709600
22  1992   694267200
23  1993   725824800
24  1994   757382400
25  1995   788940000
26  1996   820497600
27  1997   852055200
28  1998   883612800
29  1999   915170400
..   ...         ...
50  2020  1577880000
51  2021  1609437600
52  2022  1640995200
53  2023  1672552800
54  2024  1704110400
55  2025  1735668000
56  2026  1767225600
57  2027  1798783200
58  2028  1830340800
59  2029  1861898400
60  2030  1893456000
61  2031  1925013600
62  2032  1956571200
63  2033  1988128800
64  2034  2019686400
65  2035  205